<h1> Uygulamalı Veri Bilimi Ödevi Veri Analiz Bölümü </h1>

<h3>İsim: Vehbi Özcan</h3>
<h3>Numara: 211220070</h3>

## Import Bölümü

In [157]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

from sklearn import model_selection
from sklearn.neighbors import KNeighborsRegressor

# from warnings import filterwarnings
# filterwarnings("ignore")


## Veriyi Okuma

In [146]:
org_data = pd.read_excel('data.xlsx')
#data = org_data.copy()
#org_data.head()


In [147]:
df = org_data.copy()
df.head()

,Bellek Hızı,Cihaz Ağırlığı,Ekran Boyutu,Ekran Kartı Bellek Tipi,Ekran Kartı Hafızası,Ekran Kartı Tipi,Ekran Panel Tipi,İşlemci Nesli,İşlemci,İşletim Sistemi,Kart Okuyucu,Aydınlatma,Maksimum İşlemci Hızı,Max Ekran Çözünürlüğü,Parmak İzi Okuyucu,Ram (Sistem Belleği),SSD Kapasitesi,Ürün Modeli,Fiyat
0,1066 MHz,NaN,10 inç,NaN,1 GB,NaN,IPS,1.Nesil,1000M,Android,NaN,0,"1,05 GHz",NaN,NaN,NaN,1 TB,Notebook,Çok Yüksek
1,1066 MHz,NaN,10 inç,NaN,1 GB,NaN,IPS,1.Nesil,1000M,Android,NaN,0,"1,05 GHz",NaN,NaN,NaN,1 TB,Notebook,Çok Yüksek
2,1066 MHz,NaN,10 inç,NaN,1 GB,NaN,IPS,1.Nesil,1000M,Android,NaN,0,"1,05 GHz",NaN,NaN,NaN,1 TB,Notebook,Orta
3,3200 MHz,1 - 2 kg,"15,6 inç",GDDR4,2 GB,Harici Ekran Kartı,LED,10. Nesil,1035G1,Windows 10 Home,Var,0,"3,6 GHz",1920 x 1080,Yok,8 GB,512 GB,Notebook,Düşük
4,3200 MHz,1 - 2 kg,"15,6 inç",GDDR5,2 GB,Harici Ekran Kartı,LED,10. Nesil,1035G1,Windows 10 Home,Yok,0,"3,6 GHz",1920 x 1080,Yok,12 GB,1 TB,Notebook,Düşük


## Preprocessing

In [148]:
# df["Bellek Hızı"].fillna(value="3200 MHz", inplace=True)
# df["Cihaz Ağırlığı"].fillna(value="1 - 2 kg", inplace=True)
# df["Ekran Boyutu"].fillna(value="15,6 inç", inplace=True)
# df["Ekran Kartı Bellek Tipi"].fillna(value="Onboard", inplace=True)
# df["Ekran Kartı Hafızası"].fillna(value="Paylaşımlı", inplace=True)
# df["Ekran Kartı Tipi"].fillna(value="Dahili Ekran Kartı", inplace=True)
# df["Ekran Panel Tipi"].fillna(value="IPS", inplace=True)
# df["İşlemci Nesli"].fillna(value="11.Nesil", inplace=True)
# df["İşlemci"].fillna(value="1135G7", inplace=True)
# df["İşletim Sistemi"].fillna(value="Yok (Free Dos)", inplace=True)
# df["Kart Okuyucu"].fillna(value="Var", inplace=True)
# df["Aydınlatma"].fillna(value=0, inplace=True)
# df["Maksimum İşlemci Hızı"].fillna(value="4,7 GHz", inplace=True)
# df["Max Ekran Çözünürlüğü"].fillna(value="1920 x 1080", inplace=True)
# df["Parmak İzi Okuyucu"].fillna(value="Yok", inplace=True)
# df["Ram (Sistem Belleği)"].fillna(value="16 GB", inplace=True)
# df["SSD Kapasitesi"].fillna(value="512 GB", inplace=True)
# df["Ürün Modeli"].fillna(value="Notebook", inplace=True)
# df["Fiyat"].fillna(value="Düşük", inplace=True)



## Boş Verileri Doldurma

In [149]:
def fill_value(df):
    for column in df.columns:
        df[column].fillna(value = df[column].mode()[0], inplace=True)
    return df

## Aykırı ve Gürültülü Verileri Düzeltme


In [150]:
def fix_value(df):
    df['İşlemci'].replace('10. Nesil Intel® Core™ i3-10110U işlemci', '10110U',inplace=True)
    df['İşlemci Nesli'].replace('10. Nesil', '10.Nesil',inplace=True)
    df['SSD Kapasitesi'].replace(' 0 GB', '0 GB',inplace=True)
    df['Ram (Sistem Belleği)'].replace('8\xa0GB', '8 GB',inplace=True)
    return df

## Normalizasyon (MinMaxScaler)

In [151]:
def df_minmaxscaler(le_df):
    scaler = MinMaxScaler()
    le_dfr = np.reshape(le_df,(-1,1))
    le_dfr = le_dfr.astype(float)
    le_dfs = scaler.fit_transform(le_dfr)
    le_dfs = le_dfs.flatten()
    return le_dfs

## Encoding (One Hot Encoder)

In [152]:
def df_encode(df):
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
    le = LabelEncoder()
    columns = df.columns.to_list()
    columns.remove("Fiyat")
    encoded_df = ohe.fit_transform(df[columns])
    le_df = le.fit_transform(df["Fiyat"])
    le_dfs = df_minmaxscaler(le_df)
    encoded_df["Fiyat"] = le_dfs
    return encoded_df

In [153]:
def preprocess(df):
    df = fill_value(df)
    df = fix_value(df)
    encoded_df = df_encode(df)
    return encoded_df

In [154]:
encoded_df = preprocess(df)
encoded_df


,Bellek Hızı_1066 MHz,Bellek Hızı_1600 MHz,Bellek Hızı_1866 MHz,Bellek Hızı_2133 MHz,Bellek Hızı_2400 MHz,Bellek Hızı_2666 MHz,Bellek Hızı_2933 MHz,Bellek Hızı_3000 MHz,Bellek Hızı_3200 MHz,Bellek Hızı_3733 MHz,...,SSD Kapasitesi_256 GB,SSD Kapasitesi_4 TB,SSD Kapasitesi_500 GB,SSD Kapasitesi_512 GB,SSD Kapasitesi_64 GB,Ürün Modeli_Notebook,Ürün Modeli_Oyun Bilgisayarları,Ürün Modeli_Ultrabook,Ürün Modeli_İkisi Bir Arada,Fiyat
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.00
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.00
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.00
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00
990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.75
991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.75
992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00


In [156]:
df[df["SSD Kapasitesi"] == " 0 GB"]
df["Ram (Sistem Belleği)"].unique()
# df.nunique()

array(['16 GB', '8 GB', '12 GB', '32 GB', '4 GB', '20 GB', '24 GB',
       '40 GB', '36 GB', '64 GB', '128 GB'], dtype=object)

## Classification 